In [1]:
import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  
import calendar
import argparse
import glob


In [2]:
output_col_names = ['lon'
,'lat'
,'variable'
,'cohort_number'
,'stand_age'
,'potential_veg'
,'current_veg'
,'community_type'
,'subtype'
,'silt_clay'  ###check this order
,'lc_state'
,'land_area'
,'cohort_area'
,'year'
,'annual_sum'
,'monthly_maximum'
,'monthly_mean'
,'monthly_minimum'
,'Jan'
,'Feb'
,'Mar'
,'Apr'
,'May'
,'Jun'
,'Jul'
,'Aug'
,'Sep'
,'Oct'
,'Nov'
,'Dec'
,'region']

In [3]:

forest_vegs = [4, 5, 6, 8, 9, 10, 11, 16, 17, 18, 19, 20, 25, 33]
forest_types = ["Boreal Forest", "Forested Boreal Wetlands", "Boreal Woodlands","Mixed Temperate Forests", 
               "Temperate Coniferous Forests", "Temperate Deciduous Forests", "Temperate Forested Wetlands", 
               "Tropical Evergreen Forests", "Tropical Forested Wetlands", "Tropical Deciduous Forests", "Xeromorphic Forests and Woodlands"
               ,"Tropical Forested Floodplains", "Temperate Forested Floodplains", "Temperate Broadleaved Evergreen Forests"]

forest_pfts = pd.DataFrame({
'current_veg':forest_vegs, 
    'forest_type':forest_types
})


In [4]:
ensemble_dir = '/group/moniergrp/TEM_Large_Ensemble/run_support_files/large_ensemble/'

region = 'region_5/'
output_dir_path = '/group/moniergrp/TEM_Large_Ensemble/output_files/' 
var = 'LAI'
intervals_standage = np.concatenate((np.arange(-1, 100, 5),np.array([124, 149, 3000])))

In [7]:
path_summary_future = pd.read_csv(output_dir_path+region+'future_path_summary.csv')
path_summary_historical = pd.read_csv(output_dir_path+region+'historical_path_summary.csv')
path_summary_pre_data = pd.read_csv(output_dir_path+region+'pre_data_path_summary.csv')
# path_summary

In [73]:
path_summary_historical

,variable,grid_group,model,time_period,path
0,NEP,130,AWIxxCMxx1xx1xxMR,pre_data,historical/AWIxxCMxx1xx1xxMR/var_out/NEP.csv130
1,NETNMIN,0,AWIxxCMxx1xx1xxMR,pre_data,historical/AWIxxCMxx1xx1xxMR/var_out/NETNMIN.csv0
2,NPP,455,AWIxxCMxx1xx1xxMR,pre_data,historical/AWIxxCMxx1xx1xxMR/var_out/NPP.csv455
3,GPP,130,AWIxxCMxx1xx1xxMR,pre_data,historical/AWIxxCMxx1xx1xxMR/var_out/GPP.csv130
4,NCE,325,AWIxxCMxx1xx1xxMR,pre_data,historical/AWIxxCMxx1xx1xxMR/var_out/NCE.csv325
...,...,...,...,...,...
765,AVAILN,520,MRIxxESM2xx0,pre_data,historical/MRIxxESM2xx0/var_out/AVAILN.csv520
766,NETNMIN,325,MRIxxESM2xx0,pre_data,historical/MRIxxESM2xx0/var_out/NETNMIN.csv325
767,LAI,390,MRIxxESM2xx0,pre_data,historical/MRIxxESM2xx0/var_out/LAI.csv390
768,NPP,325,MRIxxESM2xx0,pre_data,historical/MRIxxESM2xx0/var_out/NPP.csv325


In [8]:
loop_set = path_summary_future.loc[path_summary_future['variable']==var].reset_index()


In [9]:
### columns to keep for analysis
keep_cols = ['model','scenario','grid_group','lon', 'lat', 'variable', 'forest_type', 'current_veg', 'year',
       'silt_clay', 'stand_age_interval_min', 'cohort_area', 'land_area',
        'monthly_mean', 'cohort_annual_change_delta', 'cohort_annual_change_percent']
### dataset for all region 1
all_future = pd.DataFrame(columns=keep_cols)

for i in range(len(loop_set)):
    ###read in cohort slice
    cohort_output = pd.read_csv(ensemble_dir + region + loop_set.loc[i, 'path']
                , names = output_col_names)

    print('cohort rows read in : '+ str(len(cohort_output)))


    # #### match pfts to the current veg, bin by standage for forests
    cohort_output=cohort_output.merge(forest_pfts, on = 'current_veg', how = 'inner')
    print('cohort rows forest only : '+ str(len(cohort_output)))

    cohort_output['stand_age_bin'] = pd.cut(
        cohort_output['stand_age'], bins=intervals_standage)
    cohort_output['stand_age_interval_min'] = cohort_output['stand_age_bin'].apply(lambda x: x.left).astype(int) + 1

    ### get cohort level change
    cohort_output['cohort_annual_prior_value'] = cohort_output.groupby(['lon', 'lat', 'cohort_number', 'current_veg'])['monthly_mean'].shift()
    cohort_output['cohort_annual_change_delta'] = cohort_output['monthly_mean'] - cohort_output['cohort_annual_prior_value']
    cohort_output['cohort_annual_change_percent'] = cohort_output['cohort_annual_change_delta'] / cohort_output['cohort_annual_prior_value']

    ### calculate temp weights for average calc
    cohort_output['temp_weight_monthly_mean'] = cohort_output['monthly_mean'] * cohort_output['cohort_area'] 
    cohort_output['temp_weight_cohort_annual_change_delta'] = cohort_output['cohort_annual_change_delta'] * cohort_output['cohort_area'] 
    cohort_output['temp_weight_cohort_annual_change_percent'] = cohort_output['cohort_annual_change_percent'] * cohort_output['cohort_area'] 
    # cohort_output[['monthly_mean', 'cohort_annual_change_delta', 'cohort_annual_change_percent']].describe()

    ### summarize by stand age bin
    cohort_output = cohort_output.groupby(
        ['lon','lat','variable','forest_type','current_veg','year','silt_clay','stand_age_interval_min']
    )[[ 'cohort_area','land_area','temp_weight_monthly_mean','temp_weight_cohort_annual_change_delta' , 'temp_weight_cohort_annual_change_percent']].sum()

    print('cohort rows post binning : '+ str(len(cohort_output)))
    cohort_output=cohort_output.reset_index()
    ### finish weighted average calculation
    cohort_output['monthly_mean'] = (cohort_output['temp_weight_monthly_mean']/cohort_output['cohort_area']).round(5)
    cohort_output['cohort_annual_change_delta'] = (cohort_output['temp_weight_cohort_annual_change_delta']/cohort_output['cohort_area']).round(5)
    cohort_output['cohort_annual_change_percent'] = (cohort_output['temp_weight_cohort_annual_change_percent']/cohort_output['cohort_area']).round(5)
    cohort_output = cohort_output.loc[:, ~cohort_output.columns.str.contains('temp_weight')]
    ### get relevant info for group, model, scenario
    cohort_output['model'] = loop_set.loc[i,'model']
    cohort_output['scenario'] = loop_set.loc[i,'scenario']
    cohort_output['grid_group'] = loop_set.loc[i,'grid_group']

    all_future = pd.concat([all_future, cohort_output])
    print(f'all future {i} : '+ str(len(all_future)))


all_future.to_csv(output_dir_path+region+var+'_future.csv'
                        ,index = False)




cohort rows read in : 753170
cohort rows forest only : 622127
cohort rows post binning : 112810
all future 0 : 112810
cohort rows read in : 915773
cohort rows forest only : 745713
cohort rows post binning : 126816
all future 1 : 239626
cohort rows read in : 818766
cohort rows forest only : 625136
cohort rows post binning : 98247
all future 2 : 337873
cohort rows read in : 979839
cohort rows forest only : 597118
cohort rows post binning : 103055
all future 3 : 440928
cohort rows read in : 468619
cohort rows forest only : 435877
cohort rows post binning : 88946
all future 4 : 529874
cohort rows read in : 812351
cohort rows forest only : 693300
cohort rows post binning : 123325
all future 5 : 653199
cohort rows read in : 918708
cohort rows forest only : 775603
cohort rows post binning : 136144
all future 6 : 789343
cohort rows read in : 597705
cohort rows forest only : 344251
cohort rows post binning : 66352
all future 7 : 855695
cohort rows read in : 891019
cohort rows forest only : 6247

/tmp/ipykernel_3124239/1299535344.py:10: DtypeWarning: Columns (14,15,16,18,19,20,21,22,23,24,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  cohort_output = pd.read_csv(ensemble_dir + region + loop_set.loc[i, 'path']


cohort rows read in : 915773
cohort rows forest only : 745713


TypeError: unsupported operand type(s) for -: 'str' and 'float'

In [13]:
cohort_output['tt'] = cohort_output['monthly_mean'].astype(float)

In [19]:
cohort_output.loc[cohort_output['tt']>100000, 'year'].unique()

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025,
       2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036,
       2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047,
       2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058,
       2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069,
       2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080,
       2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091,
       2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100])

In [20]:
cohort_output.loc[cohort_output['tt']<100000, 'tt'].describe()

count    721910.000000
mean          4.671371
std         308.650585
min           0.000000
25%           1.690000
50%           3.230000
75%           4.140000
max       99818.650000
Name: tt, dtype: float64

In [21]:
loop_set = path_summary_historical.loc[path_summary_historical['variable']==var].reset_index()
del all_future
### columns to keep for analysis
keep_cols = ['model','grid_group','lon', 'lat', 'variable', 'forest_type', 'current_veg', 'year',
       'silt_clay', 'stand_age_interval_min', 'cohort_area', 'land_area',
        'monthly_mean', 'cohort_annual_change_delta', 'cohort_annual_change_percent']
### dataset for all region 1
all_historical = pd.DataFrame(columns=keep_cols)

for i in range(len(loop_set)):
    ###read in cohort slice
    cohort_output = pd.read_csv(ensemble_dir + region + loop_set.loc[i, 'path']
                , names = output_col_names)

    # print('cohort rows read in : '+ str(len(cohort_output)))


    # #### match pfts to the current veg, bin by standage for forests
    cohort_output=cohort_output.merge(forest_pfts, on = 'current_veg', how = 'inner')
    # print('cohort rows forest only : '+ str(len(cohort_output)))

    cohort_output['stand_age_bin'] = pd.cut(
        cohort_output['stand_age'], bins=intervals_standage)
    cohort_output['stand_age_interval_min'] = cohort_output['stand_age_bin'].apply(lambda x: x.left).astype(int) + 1
    ###flag potential problems
    cohort_output['monthly_mean'] = cohort_output['monthly_mean'].astype(float)
    cohort_output['issue'] = 1*cohort_output['monthly_mean'] > 100000

    ### get cohort level change
    cohort_output['cohort_annual_prior_value'] = cohort_output.groupby(['lon', 'lat', 'cohort_number', 'current_veg'])['monthly_mean'].shift()
    cohort_output['cohort_annual_change_delta'] = cohort_output['monthly_mean'] - cohort_output['cohort_annual_prior_value']
    cohort_output['cohort_annual_change_percent'] = cohort_output['cohort_annual_change_delta'] / cohort_output['cohort_annual_prior_value']

    ### calculate temp weights for average calc
    cohort_output['temp_weight_monthly_mean'] = cohort_output['monthly_mean'] * cohort_output['cohort_area'] 
    cohort_output['temp_weight_cohort_annual_change_delta'] = cohort_output['cohort_annual_change_delta'] * cohort_output['cohort_area'] 
    cohort_output['temp_weight_cohort_annual_change_percent'] = cohort_output['cohort_annual_change_percent'] * cohort_output['cohort_area'] 
    # cohort_output[['monthly_mean', 'cohort_annual_change_delta', 'cohort_annual_change_percent']].describe()

    ### summarize by stand age bin
    cohort_output = cohort_output.groupby(
        ['lon','lat','variable','forest_type','current_veg','year','silt_clay','stand_age_interval_min', 'issue']
    )[[ 'cohort_area','land_area','temp_weight_monthly_mean','temp_weight_cohort_annual_change_delta' , 'temp_weight_cohort_annual_change_percent']].sum()

    # print('cohort rows post binning : '+ str(len(cohort_output)))
    cohort_output=cohort_output.reset_index()
    ### finish weighted average calculation
    cohort_output['monthly_mean'] = (cohort_output['temp_weight_monthly_mean']/cohort_output['cohort_area']).round(5)
    cohort_output['cohort_annual_change_delta'] = (cohort_output['temp_weight_cohort_annual_change_delta']/cohort_output['cohort_area']).round(5)
    cohort_output['cohort_annual_change_percent'] = (cohort_output['temp_weight_cohort_annual_change_percent']/cohort_output['cohort_area']).round(5)
    cohort_output = cohort_output.loc[:, ~cohort_output.columns.str.contains('temp_weight')]
    ### get relevant info for group, model, scenario
    cohort_output['model'] = loop_set.loc[i,'model']
    cohort_output['grid_group'] = loop_set.loc[i,'grid_group']

    all_historical = pd.concat([all_historical, cohort_output])
    print(f'all historical {i} : '+ str(len(all_historical)))


all_historical.to_csv(output_dir_path+region+var+'_historical.csv'
                        ,index = False)




all historical 0 : 63111
all historical 1 : 124512
all historical 2 : 177589
all historical 3 : 232549
all historical 4 : 256765
all historical 5 : 316715
all historical 6 : 385491
all historical 7 : 423502
all historical 8 : 492685
all historical 9 : 562477
all historical 10 : 625588
all historical 11 : 678665
all historical 12 : 740066
all historical 13 : 764282
all historical 14 : 824232
all historical 15 : 893008
all historical 16 : 931019
all historical 17 : 985979
all historical 18 : 1055162
all historical 19 : 1124954
all historical 20 : 1188065
all historical 21 : 1249466


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [66]:
loop_set = path_summary_pre_data.loc[path_summary_pre_data['variable']==var].reset_index()
del all_historical
### columns to keep for analysis
keep_cols = ['model','grid_group','lon', 'lat', 'variable', 'forest_type', 'current_veg', 'year',
       'silt_clay', 'stand_age_interval_min', 'cohort_area', 'land_area',
        'monthly_mean', 'cohort_annual_change_delta', 'cohort_annual_change_percent']
### dataset for all region 1
all_pre_data = pd.DataFrame(columns=keep_cols)

for i in range(len(loop_set)):
    ###read in cohort slice
    cohort_output = pd.read_csv(ensemble_dir + region + loop_set.loc[i, 'path']
                , names = output_col_names)

    print('cohort rows read in : '+ str(len(cohort_output)))


    # #### match pfts to the current veg, bin by standage for forests
    cohort_output=cohort_output.merge(forest_pfts, on = 'current_veg', how = 'inner')
    print('cohort rows forest only : '+ str(len(cohort_output)))

    cohort_output['stand_age_bin'] = pd.cut(
        cohort_output['stand_age'], bins=intervals_standage)
    cohort_output['stand_age_interval_min'] = cohort_output['stand_age_bin'].apply(lambda x: x.left).astype(int) + 1

    ### get cohort level change
    cohort_output['cohort_annual_prior_value'] = cohort_output.groupby(['lon', 'lat', 'cohort_number', 'current_veg'])['monthly_mean'].shift()
    cohort_output['cohort_annual_change_delta'] = cohort_output['monthly_mean'] - cohort_output['cohort_annual_prior_value']
    cohort_output['cohort_annual_change_percent'] = cohort_output['cohort_annual_change_delta'] / cohort_output['cohort_annual_prior_value']

    ### calculate temp weights for average calc
    cohort_output['temp_weight_monthly_mean'] = cohort_output['monthly_mean'] * cohort_output['cohort_area'] 
    cohort_output['temp_weight_cohort_annual_change_delta'] = cohort_output['cohort_annual_change_delta'] * cohort_output['cohort_area'] 
    cohort_output['temp_weight_cohort_annual_change_percent'] = cohort_output['cohort_annual_change_percent'] * cohort_output['cohort_area'] 
    # cohort_output[['monthly_mean', 'cohort_annual_change_delta', 'cohort_annual_change_percent']].describe()

    ### summarize by stand age bin
    cohort_output = cohort_output.groupby(
        ['lon','lat','variable','forest_type','current_veg','year','silt_clay','stand_age_interval_min']
    )[[ 'cohort_area','land_area','temp_weight_monthly_mean','temp_weight_cohort_annual_change_delta' , 'temp_weight_cohort_annual_change_percent']].sum()

    print('cohort rows post binning : '+ str(len(cohort_output)))
    cohort_output=cohort_output.reset_index()
    ### finish weighted average calculation
    cohort_output['monthly_mean'] = (cohort_output['temp_weight_monthly_mean']/cohort_output['cohort_area']).round(5)
    cohort_output['cohort_annual_change_delta'] = (cohort_output['temp_weight_cohort_annual_change_delta']/cohort_output['cohort_area']).round(5)
    cohort_output['cohort_annual_change_percent'] = (cohort_output['temp_weight_cohort_annual_change_percent']/cohort_output['cohort_area']).round(5)
    cohort_output = cohort_output.loc[:, ~cohort_output.columns.str.contains('temp_weight')]
    ### get relevant info for group, model, scenario
    cohort_output['model'] = loop_set.loc[i,'model']
    cohort_output['grid_group'] = loop_set.loc[i,'grid_group']

    all_pre_data = pd.concat([all_pre_data, cohort_output])
    print(f'all pre data {i} : '+ str(len(all_pre_data)))


all_pre_data.to_csv(output_dir_path+region+var+'_pre_data.csv'
                        ,index = False)




cohort rows read in : 77140
cohort rows forest only : 23196
cohort rows post binning : 22832
all pre data 0 : 22832
cohort rows read in : 25413
cohort rows forest only : 25413
cohort rows post binning : 23983
all pre data 1 : 46815
cohort rows read in : 33214
cohort rows forest only : 33214
cohort rows post binning : 27437
all pre data 2 : 74252
cohort rows read in : 180830
cohort rows forest only : 31642
cohort rows post binning : 23783
all pre data 3 : 98035
cohort rows read in : 145529
cohort rows forest only : 37854
cohort rows post binning : 26303
all pre data 4 : 124338
cohort rows read in : 22815
cohort rows forest only : 22815
cohort rows post binning : 22815
all pre data 5 : 147153
cohort rows read in : 29987
cohort rows forest only : 23166
cohort rows post binning : 22815
all pre data 6 : 169968
cohort rows read in : 214731
cohort rows forest only : 32809
cohort rows post binning : 24039
all pre data 7 : 194007
cohort rows read in : 43966
cohort rows forest only : 39915
cohor

In [68]:
cohort_output

,lon,lat,variable,forest_type,current_veg,year,silt_clay,stand_age_interval_min,cohort_area,land_area,monthly_mean,cohort_annual_change_delta,cohort_annual_change_percent,model,grid_group
0,-177.0,-44.0,GPP,Temperate Broadleaved Evergreen Forests,33,1500,71.21,150,991,991,153.85,0.00,0.00000,MRIxxESM2xx0,0
1,-177.0,-44.0,GPP,Temperate Broadleaved Evergreen Forests,33,1501,71.21,150,991,991,158.54,4.69,0.03048,MRIxxESM2xx0,0
2,-177.0,-44.0,GPP,Temperate Broadleaved Evergreen Forests,33,1502,71.21,150,991,991,166.10,7.56,0.04769,MRIxxESM2xx0,0
3,-177.0,-44.0,GPP,Temperate Broadleaved Evergreen Forests,33,1503,71.21,150,991,991,170.31,4.21,0.02535,MRIxxESM2xx0,0
4,-177.0,-44.0,GPP,Temperate Broadleaved Evergreen Forests,33,1504,71.21,150,991,991,174.94,4.63,0.02719,MRIxxESM2xx0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22827,-158.5,63.0,GPP,Forested Boreal Wetlands,5,1846,59.02,150,1390,1391,38.53,0.40,0.01049,MRIxxESM2xx0,0
22828,-158.5,63.0,GPP,Forested Boreal Wetlands,5,1847,59.02,150,1390,1391,35.08,-3.45,-0.08954,MRIxxESM2xx0,0
22829,-158.5,63.0,GPP,Forested Boreal Wetlands,5,1848,59.02,150,1390,1391,41.04,5.96,0.16990,MRIxxESM2xx0,0
22830,-158.5,63.0,GPP,Forested Boreal Wetlands,5,1849,59.02,150,1390,1391,33.84,-7.20,-0.17544,MRIxxESM2xx0,0


In [71]:
cohort_output = pd.read_csv(ensemble_dir + region + loop_set.loc[i, 'path']
                , names = output_col_names)

print('cohort rows read in : '+ str(len(cohort_output)))


# #### match pfts to the current veg, bin by standage for forests
cohort_output=cohort_output.merge(forest_pfts, on = 'current_veg', how = 'inner')
print('cohort rows forest only : '+ str(len(cohort_output)))

cohort_output['stand_age_bin'] = pd.cut(
    cohort_output['stand_age'], bins=intervals_standage)
cohort_output['stand_age_interval_min'] = cohort_output['stand_age_bin'].apply(lambda x: x.left).astype(int) + 1

### get cohort level change
cohort_output['cohort_annual_prior_value'] = cohort_output.groupby(['lon', 'lat', 'cohort_number', 'current_veg'])['monthly_mean'].shift().round(4)
cohort_output['cohort_annual_change_delta'] = cohort_output['monthly_mean'] - cohort_output['cohort_annual_prior_value']
cohort_output['cohort_annual_change_percent'] = cohort_output['cohort_annual_change_delta'] / cohort_output['cohort_annual_prior_value']
cohort_output[['monthly_mean', 'cohort_annual_change_delta', 'cohort_annual_prior_value']]


cohort rows read in : 77140
cohort rows forest only : 23196


,monthly_mean,cohort_annual_change_delta,cohort_annual_prior_value
0,153.85,NaN,NaN
1,158.54,4.69,153.85
2,166.10,7.56,158.54
3,170.31,4.21,166.10
4,174.94,4.63,170.31
...,...,...,...
23191,38.53,0.40,38.13
23192,35.08,-3.45,38.53
23193,41.04,5.96,35.08
23194,33.84,-7.20,41.04


,monthly_mean,cohort_annual_change_delta,cohort_annual_prior_value
0,153.85,NaN,NaN
1,158.54,4.69,153.85
2,166.10,7.56,158.54
3,170.31,4.21,166.10
4,174.94,4.63,170.31
...,...,...,...
23191,38.53,0.40,38.13
23192,35.08,-3.45,38.53
23193,41.04,5.96,35.08
23194,33.84,-7.20,41.04


In [42]:
cohort_output = cohort_output.groupby(
    ['lon','lat','variable','forest_type','current_veg','year','silt_clay','stand_age_interval_min']
)[[ 'cohort_area','land_area','temp_weight_monthly_mean','temp_weight_cohort_annual_change_delta' , 'temp_weight_cohort_annual_change_percent']].sum()

cohort_output['monthly_mean'] = cohort_output['temp_weight_monthly_mean']/cohort_output['cohort_area']
cohort_output['cohort_annual_change_delta'] = cohort_output['temp_weight_cohort_annual_change_delta']/cohort_output['cohort_area']
cohort_output['cohort_annual_change_percent'] = cohort_output['temp_weight_cohort_annual_change_percent']/cohort_output['cohort_area']


cohort_output[['monthly_mean', 'cohort_annual_change_delta', 'cohort_annual_change_percent']].describe()


,monthly_mean,cohort_annual_change_delta,cohort_annual_change_percent
count,35131.000000,35131.000000,35131.000000
mean,11.825025,0.272804,0.106155
std,12.961677,3.639155,0.892140
min,0.020000,-48.270000,-0.998822
25%,2.015000,-0.280000,-0.073143
50%,2.390000,0.024000,0.014060
75%,22.110833,0.422500,0.152924
max,104.060000,51.100000,144.375000


In [23]:
cohort_output.loc[cohort_output.cohort_annual_change_percent<-.9, 'year']

74295    2030
95336    2068
Name: year, dtype: int64

In [31]:
cohort_output.forest_type.unique()

array(['Boreal Forest', 'Forested Boreal Wetlands'], dtype=object)